In [2]:
import numpy as np
import pandas
import pandas as pd

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-18 07:50:46--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  2.33MB/s    in 0.6s    

2022-09-18 07:50:47 (2.33 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [4]:
df = pd.read_csv('housing.csv')

In [5]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [7]:
column_names = ['latitude', 'longitude', 'housing_median_age',
                'total_rooms', 'total_bedrooms', 'population',
                'households', 'median_income', 'median_house_value']

In [9]:
df = df[column_names]

In [10]:
df

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


### Question 1

Find a feature with missing values. How many missing values does it have?


In [21]:
na = df.isna().sum().loc[lambda x: x> 0]

In [31]:
list(na)[0]

207

### Question 2

What's the median (50% percentile) for variable 'population'?

In [34]:
df.population.median()

1166.0

### Split the data

* Shuffle the initial dataset, use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Make sure that the target value ('median_house_value') is not in your dataframe.
* Apply the log transformation to the median_house_value variable using the `np.log1p()` function.


In [71]:
n = df.shape[0]
n_test = int(n * 0.2)
n_val = int(n * 0.2)
n_train = n - n_val - n_test

In [72]:
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]


In [73]:
df_list = [df_train, df_val, df_test]

In [74]:
n_test, n_val, n_train

(4128, 4128, 12384)

In [75]:
y_train = list(df_train.median_house_value)
y_test = list(df_test.median_house_value)
y_val = list(df_val.median_house_value)
y_list = [y_train, y_val, y_test]

In [76]:
for _df in df_list:
    del _df['median_house_value']

In [79]:
y_val = np.log1p(y_val)
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:
- With 0
- With mean
- Both are equally goodm

In [81]:
def dot(xi, w):
    n = len(xi)

    res = 0.0

    for j in range(n):
        res = res + xi[j] * w[j]

    return res

In [82]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)

    return w_full[0], w_full[1:]

In [155]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [93]:
def prepare_with_zero(df):
    return df.fillna(0).values

In [98]:
def calc_rmse(method_name, df_train, y_train, df_val, y_val):
    X_train = method_name(df_train)
    train_w0, train_w = train_linear_regression(X_train, y_train)
    X_val = method_name(df_val)
    y_pred = train_w0 + X_val.dot(train_w)
    return rmse(y_val, y_pred)


In [99]:
calc_rmse(prepare_with_zero, df_train, y_train, df_val, y_val)

0.32953303652313454

In [120]:
def prepare_with_mean(df):
    return df.fillna(df.mean()).values

In [121]:
print(calc_rmse(prepare_with_zero, df_train, y_train, df_val, y_val))
print(calc_rmse(prepare_with_mean, df_train, y_train, df_val, y_val))

0.32953303652313454
0.3290195439004034


### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0.
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:
- 0
- 0.000001
- 0.001
- 0.0001

In [156]:
def calc_rmse_v2(na_fill_func, df_train, y_train, df_val, y_val, r = 0):
    X_train = na_fill_func(df_train)
    train_w0, train_w = train_linear_regression_reg(X_train, y_train, r)
    X_val = na_fill_func(df_val)
    y_pred = train_w0 + X_val.dot(train_w)
    return rmse(y_val, y_pred)

In [136]:
regression_list = [0, 0.000001, 0.0001, 0.001, 0.1, 1, 5, 10]

In [137]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)

    return w_full[0], w_full[1:]

In [138]:
calc_rmse_v2(prepare_with_zero, df_train, y_train, df_val, y_val, r = 0)

0.32953303652313454

In [159]:
d = {}
for r in regression_list:
    rmse_v = round(calc_rmse_v2(prepare_with_zero, df_train, y_train, df_val, y_val, r), 2)
    if rmse_v in d:
        d[rmse_v].append(r)
    else:
        d.setdefault(round(calc_rmse_v2(prepare_with_zero, df_train, y_train, df_val, y_val, r), 2), [r])

In [166]:
sorted(d[sorted(d)[0]])[0]

0


* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

> Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different.
> If standard deviation of scores is low, then our model is *stable*.

Options:
- 0.16
- 0.00005
- 0.005
- 0.15555

In [172]:
seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 42]
rmse_list = []
for seed_val in seed_values:
    idx = np.arange(n)
    np.random.seed(seed_val)
    np.random.shuffle(idx)
    df_train = df.iloc[idx[:n_train]]
    df_val = df.iloc[idx[n_train:n_train+n_val]]
    df_test = df.iloc[idx[n_train+n_val:]]
    df_list = [df_train, df_val, df_test]

    y_train = list(df_train.median_house_value)
    y_test = list(df_test.median_house_value)
    y_val = list(df_val.median_house_value)
    y_list = [y_train, y_val, y_test]
    for _df in df_list:
        del _df['median_house_value']
    y_val = np.log1p(y_val)
    y_train = np.log1p(y_train)
    y_test = np.log1p(y_test)
    rmse_list.append(calc_rmse_v2(prepare_with_zero, df_train, y_train, df_val, y_val, r))

In [173]:
rmse_list

[0.34788411528721824,
 0.34839195191549355,
 0.3425754849256515,
 0.35036342462791126,
 0.35086747548756453,
 0.3546539619167348,
 0.35601733893278675,
 0.34648118435721603,
 0.35586217752994537,
 0.34576857182082693,
 0.34060638078084005]

In [174]:
np.std(rmse_list)

0.00489670262940542

### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`.
* What's the RMSE on the test dataset?

Options:
- 0.35
- 0.135
- 0.450
- 0.245

In [175]:
np.random.seed(9)

In [178]:
idx = np.arange(n)
np.random.shuffle(idx)
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]
df_list = [df_train, df_val, df_test]

y_train = list(df_train.median_house_value)
y_test = list(df_test.median_house_value)
y_val = list(df_val.median_house_value)
y_list = [y_train, y_val, y_test]
for _df in df_list:
    del _df['median_house_value']
y_val = np.log1p(y_val)
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)


In [191]:
calc_rmse_v2(prepare_with_zero, pd.concat([df_train, df_val]), np.concatenate([y_train, y_val]), df_test, y_test, r)

0.3451463090784051